In [ ]:
"""
NOTES:
    - DraftsActive will be available as soon as the draft is entered regardless
    of fill status.
    - DraftDetail picks 

TODO:
    - A shell of the draft will need to be created at the beginning in order
    to derive the # of picks between selections for each user/pick
        - Add a 'users' and 'draft_entries' df to DraftDetail which pulls the 
        data found in the url_draft url and sliced using ['draft']['users']
        and ['draft']['draft_entries']
        - Will want to wait until the status (i.e. ['draft']['status'] == 'drafting')
        - Need to update DraftsDetail to account for IndexError that occurs
        when creating the main df at the very beginning of the draft when
        'picks' is empty


"""

In [1]:
from os.path import join
import pickle

import pandas as pd
import numpy as np
import getpass

import UD_draft_model.scrapers.scrape_site.scrape_league_data as scrape_site
import UD_draft_model.scrapers.scrape_site.pull_bearer_token as pb
import UD_draft_model.data_processing.prepare_drafts as prepare_drafts
import UD_draft_model.data_processing.add_features as add_features

In [2]:
import getpass


pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)

### Variables to change ###
chromedriver_path = "/usr/bin/chromedriver"
username = input("Enter Underdog username: ")
password = getpass.getpass()

### Keep as is ###
url = "https://underdogfantasy.com/lobby"
bearer_token = pb.pull_bearer_token(url, chromedriver_path, username, password)

In [9]:
player_vars = [
    'appearance_id', 'player_id', 'position', 'first_name', 'last_name',
    'abbr', 'team_name', 'adp', 'season_projected_points'
]

active_drafts = scrape_site.DraftsActive(bearer_token)

df = active_drafts.create_df_active_drafts()

slate_id = df['slate_id'].iloc[0]
scoring_type_id = df['scoring_type_id'].iloc[0]
draft_id = [df['id'].iloc[0]]
rounds = df['rounds'].iloc[0]
refs = scrape_site.ReferenceData(slate_id, scoring_type_id)

draft_detail = scrape_site.DraftsDetail(draft_id, bearer_token)

df_players = refs.create_df_players_master()
df_players = df_players[player_vars]
df_draft = draft_detail.create_df_drafts()
df_entries = draft_detail.create_df_draft_entries()

# url = [url for url in draft_detail.url_drafts.values()][0]
# header = draft_detail.auth_header
# header['authorization'] = bearer_token
# _df = draft_detail.read_in_site_data(url, headers=header)


# df
df_players

,appearance_id,player_id,position,first_name,last_name,abbr,team_name,adp,season_projected_points
0,0e383613-0c4f-4f3c-8145-73c0863a1929,d61fb310-35c9-4f90-a99c-fb4605a870f4,WR,Justin,Jefferson,MIN,Minnesota Vikings,1.2,288.7
1,c307679c-8ee9-47e3-882f-f386bc00f510,19818cc6-e21b-4258-9315-93cb64fb92e7,WR,Ja'Marr,Chase,CIN,Cincinnati Bengals,2.4,271.7
2,0a57b8db-0a17-4b0b-bab2-cfadb788d9b8,cb7b1dc6-91ce-47b8-813c-07486371a922,RB,Christian,McCaffrey,SF,San Francisco 49ers,3.0,303.5
3,e32a0878-ba15-4fb1-8518-d86663bbaa17,c5719ef0-3dcc-4740-84af-ef600d160bd9,WR,Cooper,Kupp,LAR,Los Angeles Rams,4.7,302.0
4,c7bc600e-e3be-48ff-86cf-3b1d0ad42a5f,9ba8d5db-3aa3-4600-98f0-406a1e67bb22,TE,Travis,Kelce,KC,Kansas City Chiefs,5.1,209.2
...,...,...,...,...,...,...,...,...,...
1559,7e7288d0-167c-43b8-b75b-1d0da9980a90,c35b6964-a0c4-48c2-9d0d-24bcc1faf218,RB,Rasheen,Ali,NaN,NaN,-,0.0
1560,7e73eedb-bed6-4693-94f1-219236448b17,bb3b8500-a070-444c-8ef8-8105f87b26ee,RB,Ryquell,Armstead,NaN,NaN,-,0.0
1561,7f22b1c9-f8b2-4c0a-b2f0-85e13ad04dba,cc88202f-4e53-44db-ad19-f21927c02848,TE,Jordan,Franks,NaN,NaN,-,0.0
1562,7f2a8bc4-6b1e-4704-8e20-64c4c1188f1f,8da84d64-bab4-4fdd-ac2d-f6729e5a31a5,TE,Kaden,Smith,NaN,NaN,-,0.0


In [18]:
def create_draft_board(df_entries: pd.DataFrame, num_rounds: int) -> pd.DataFrame:
    """" 
    Creates a df of all user/pick numbers given the entries of a snake draft.
    """

    df_asc = df_entries.sort_values(by='pick_order')
    df_desc = df_entries.sort_values(by='pick_order', ascending=False)

    dfs = []
    for round in range(1, num_rounds + 1):
        if round % 2 == 0:
            df_round = df_desc.copy()
        else:
            df_round = df_asc.copy()

        df_round['round'] = round

        dfs.append(df_round)

    df = pd.concat(dfs).reset_index(drop=True)
    df['number'] = df.index + 1
    df.rename(columns={'id': 'draft_entry_id'}, inplace=True)

    keep_vars = [
        'draft_id', 'draft_entry_id', 'user_id', 'username', 'round', 'number'
    ]

    df = df[keep_vars]

    return df


def update_board(df_board: pd.DataFrame, df_draft: pd.DataFrame) -> pd.DataFrame:
    """ 
    Updates the draft board with the selections made.
    """

    df = pd.merge(
        df_board, df_draft, 
        how='left', on=['draft_id', 'draft_entry_id', 'number']
    )

    return df


def get_current_pick(df_board: pd.DataFrame) -> pd.DataFrame:
    """ 
    Filters the draft board down to the current pick.
    """

    df = df_board.loc[df_board['appearance_id'].isnull()].iloc[0:1]

    return df


def get_avail_players(df_players: pd.DataFrame, df_draft: pd.DataFrame) -> pd.DataFrame:
    """ 
    Pulls the remaining available players to draft.
    """

    df = (
        df_players
        .loc[~df_players['appearance_id'].isin(df_draft['appearance_id'])]
    )

    return df


df_board = create_draft_board(df_entries, rounds)
df_board = prepare_drafts.add_next_pick_number(df_board)

df_board = update_board(df_board, df_draft)

df_avail_players = get_avail_players(df_players, df_draft)

,draft_id,draft_entry_id,user_id,username,round,number,next_pick_number,id,appearance_id,created_at,pick_slot_id,points,projection_adp,projection_points,swapped
155,317e9315-c5cf-415e-bc6c-6a607b57747e,31e108e4-8fa5-46d4-8e49-cf2dc0a32bc3,5174dcf3-9c9d-44e0-99d8-716f1d59e0aa,DUTCHBOY241,13,156,180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# _df = prepare_drafts.add_next_pick_number(df_board)

_df = get_current_pick(df_board)

_df
df_avail_players

# _df
# df


,appearance_id,player_id,position,first_name,last_name,abbr,team_name,adp,season_projected_points
136,9893b45f-50f7-4a0a-afd0-5412c42aa0bb,2b137246-eb40-4e14-adf0-7f0287d59ba4,WR,Allen,Lazard,GB,Green Bay Packers,139.4,132.7
140,f1e93a06-8c39-42a9-b700-db30125d7942,620f78f4-a38c-491d-9088-2a7b9e5dbbae,RB,Sean,Tucker,NaN,NaN,141.3,0.0
143,ff85f1ae-5a5a-4796-903c-2426e86bf6b0,8787ffdd-7fc0-4f90-9f40-c1ab2e7692fe,WR,Romeo,Doubs,GB,Green Bay Packers,144.4,116.2
144,05662cdf-40b2-4830-8cc6-2ee71d91dabb,1a783937-492f-47a2-9058-574dc3396d8d,WR,Isaiah,Hodgins,NYG,NY Giants,144.6,108.3
147,116f8f55-31ce-467d-a42b-bcb7b3bbfa45,59614625-f243-4cb9-8038-146619f12e70,QB,CJ,Stroud,NaN,NaN,148.8,0.0
...,...,...,...,...,...,...,...,...,...
1559,7e7288d0-167c-43b8-b75b-1d0da9980a90,c35b6964-a0c4-48c2-9d0d-24bcc1faf218,RB,Rasheen,Ali,NaN,NaN,-,0.0
1560,7e73eedb-bed6-4693-94f1-219236448b17,bb3b8500-a070-444c-8ef8-8105f87b26ee,RB,Ryquell,Armstead,NaN,NaN,-,0.0
1561,7f22b1c9-f8b2-4c0a-b2f0-85e13ad04dba,cc88202f-4e53-44db-ad19-f21927c02848,TE,Jordan,Franks,NaN,NaN,-,0.0
1562,7f2a8bc4-6b1e-4704-8e20-64c4c1188f1f,8da84d64-bab4-4fdd-ac2d-f6729e5a31a5,TE,Kaden,Smith,NaN,NaN,-,0.0


In [51]:
# def create_draft_board(df: pd.DataFrame, num_rounds: int) -> pd.DataFrame:
#     """" 
#     Creates a df of all user/pick numbers given the entries of a draft.
#     """

#     df_asc = df.sort_values(by='pick_order')
#     df_desc = df.sort_values(by='pick_order', ascending=False)

#     dfs = []
#     for round in range(1, num_rounds + 1):
#         if round % 2 == 0:
#             df_round = df_desc.copy()
#         else:
#             df_round = df_asc.copy()

#         df_round['round'] = round

#         dfs.append(df_round)

#     df = pd.concat(dfs).reset_index(drop=True)
#     df['number'] = df.index + 1
#     df.rename(columns={'id': 'draft_entry_id'}, inplace=True)

#     keep_vars = [
#         'draft_id', 'draft_entry_id', 'user_id', 'username', 'round', 'number'
#     ]

#     df = df[keep_vars]

#     return df

pd.set_option('display.max_rows', 30)
df_board = create_draft_board(df_entries, rounds)

df_board

,draft_id,draft_entry_id,user_id,username,round,number
0,317e9315-c5cf-415e-bc6c-6a607b57747e,7d55e3d8-5168-46be-ba6e-5edbd651ed93,03717131-e053-451d-9be6-09755f5875ae,CONNORDELONG,1,1
1,317e9315-c5cf-415e-bc6c-6a607b57747e,d2f0c07d-88ef-41b6-bb80-0caa7585a847,6cf9cd0c-d74f-4942-8bc0-572bc3518a3b,FRESHLYBAKED,1,2
2,317e9315-c5cf-415e-bc6c-6a607b57747e,ec5fd82a-3203-4286-8bd9-5ee079652d73,76f9314e-4c41-4507-8bd5-fcb631d82f73,IAMSCAM561,1,3
3,317e9315-c5cf-415e-bc6c-6a607b57747e,111784cb-ea10-4ba8-9ee1-f7076497c808,d747d4fa-d8cb-45f6-8353-dc4c2257ff38,KATABS,1,4
4,317e9315-c5cf-415e-bc6c-6a607b57747e,3ab3c61d-adf3-4d39-b8de-d9e0080dbcc1,10e759f3-d633-4003-a75d-6634b1287696,FISHE2JS,1,5
...,...,...,...,...,...,...
235,317e9315-c5cf-415e-bc6c-6a607b57747e,3ab3c61d-adf3-4d39-b8de-d9e0080dbcc1,10e759f3-d633-4003-a75d-6634b1287696,FISHE2JS,20,236
236,317e9315-c5cf-415e-bc6c-6a607b57747e,111784cb-ea10-4ba8-9ee1-f7076497c808,d747d4fa-d8cb-45f6-8353-dc4c2257ff38,KATABS,20,237
237,317e9315-c5cf-415e-bc6c-6a607b57747e,ec5fd82a-3203-4286-8bd9-5ee079652d73,76f9314e-4c41-4507-8bd5-fcb631d82f73,IAMSCAM561,20,238
238,317e9315-c5cf-415e-bc6c-6a607b57747e,d2f0c07d-88ef-41b6-bb80-0caa7585a847,6cf9cd0c-d74f-4942-8bc0-572bc3518a3b,FRESHLYBAKED,20,239


In [48]:
_df = pd.merge(
    df_board, df_draft, how='left', on=['draft_id', 'draft_entry_id', 'number']
)

df_draft

,id,appearance_id,created_at,draft_entry_id,number,pick_slot_id,points,projection_adp,projection_points,swapped,draft_id
0,8b35d101-816f-47d7-99fb-2de04d9b1113,0e383613-0c4f-4f3c-8145-73c0863a1929,2023-02-25T19:01:43Z,7d55e3d8-5168-46be-ba6e-5edbd651ed93,1,c167e7db-a42f-40f2-a364-34aadb768ada,0.00,1.2,323.2,False,317e9315-c5cf-415e-bc6c-6a607b57747e
1,4d03b1fe-7329-46a9-91be-d9a2a3b7bb50,c307679c-8ee9-47e3-882f-f386bc00f510,2023-02-25T19:13:42Z,d2f0c07d-88ef-41b6-bb80-0caa7585a847,2,c167e7db-a42f-40f2-a364-34aadb768ada,0.00,2.5,288.3,False,317e9315-c5cf-415e-bc6c-6a607b57747e
2,59e0ae17-b1f9-4714-8401-0801d62b3ec4,0a57b8db-0a17-4b0b-bab2-cfadb788d9b8,2023-02-25T19:17:43Z,ec5fd82a-3203-4286-8bd9-5ee079652d73,3,8b25d7d2-613b-4a5a-b47b-a6ea125841c0,0.00,3.0,303.5,False,317e9315-c5cf-415e-bc6c-6a607b57747e
3,e93a159c-1593-43c9-931b-41a9605c9b90,e32a0878-ba15-4fb1-8518-d86663bbaa17,2023-02-25T19:18:30Z,111784cb-ea10-4ba8-9ee1-f7076497c808,4,c167e7db-a42f-40f2-a364-34aadb768ada,0.00,4.9,302.0,False,317e9315-c5cf-415e-bc6c-6a607b57747e
4,e6be1fb2-cf0e-4705-82f7-5db524772076,c7bc600e-e3be-48ff-86cf-3b1d0ad42a5f,2023-02-25T19:25:11Z,3ab3c61d-adf3-4d39-b8de-d9e0080dbcc1,5,16a20025-458b-4d97-a2cf-4c6516e040d1,0.00,4.7,273.0,False,317e9315-c5cf-415e-bc6c-6a607b57747e
...,...,...,...,...,...,...,...,...,...,...,...
120,9cd80fe0-1b0f-4ac2-951b-3394b7314607,53b1c5fd-2045-45c4-80ed-dceb6c036cbd,2023-03-05T16:08:59Z,7d55e3d8-5168-46be-ba6e-5edbd651ed93,121,6c82efa9-a893-4991-bbea-6135df5b2118,0.00,131.6,133.2,False,317e9315-c5cf-415e-bc6c-6a607b57747e
121,40eb323d-c1a9-418d-b0cd-8b067117bb1d,5347acfe-6105-4c2d-850d-89ce6f5e9657,2023-03-05T17:00:24Z,d2f0c07d-88ef-41b6-bb80-0caa7585a847,122,6c82efa9-a893-4991-bbea-6135df5b2118,0.00,145.3,126.6,False,317e9315-c5cf-415e-bc6c-6a607b57747e
122,a5ccc6df-a27c-4f83-a1fc-42266e74d79d,81685c18-1ff8-4090-abf4-9e335c6f7afc,2023-03-05T17:54:22Z,ec5fd82a-3203-4286-8bd9-5ee079652d73,123,6c82efa9-a893-4991-bbea-6135df5b2118,0.00,129.0,157.4,False,317e9315-c5cf-415e-bc6c-6a607b57747e
123,a01594c5-76a6-4d0b-83ba-388d8ab81de7,ab803760-261b-4ae8-b140-87b55659812e,2023-03-05T17:55:15Z,111784cb-ea10-4ba8-9ee1-f7076497c808,124,5c8b3cd4-9505-4713-b74d-913ed88a8282,0.00,122.0,281.0,False,317e9315-c5cf-415e-bc6c-6a607b57747e


In [49]:
df_players

,appearance_id,badges,lineup_status_id,match_id,match_type,player_id,position_id,adp,season_projected_points,position_rank,position,country,first_name,last_name,sport_id,team_id,abbr,team_name
0,0e383613-0c4f-4f3c-8145-73c0863a1929,[],None,1164,Season,d61fb310-35c9-4f90-a99c-fb4605a870f4,ddd5757d-e6f2-5bdc-9d42-4fe825a2eaed,1.2,288.7,WR1,WR,None,Justin,Jefferson,NFL,5af1e185-627e-5345-851d-3c65c5b66614,MIN,Minnesota Vikings
1,c307679c-8ee9-47e3-882f-f386bc00f510,[],None,1164,Season,19818cc6-e21b-4258-9315-93cb64fb92e7,ddd5757d-e6f2-5bdc-9d42-4fe825a2eaed,2.5,271.7,WR2,WR,None,Ja'Marr,Chase,NFL,ab235cf0-a041-5d36-8241-90a90f0dcb5e,CIN,Cincinnati Bengals
2,0a57b8db-0a17-4b0b-bab2-cfadb788d9b8,[],None,1164,Season,cb7b1dc6-91ce-47b8-813c-07486371a922,e2571131-7600-5bc9-878b-946a992c8203,2.9,303.5,RB1,RB,None,Christian,McCaffrey,NFL,7161e62b-de20-56e2-a300-0dc23637faaa,SF,San Francisco 49ers
3,e32a0878-ba15-4fb1-8518-d86663bbaa17,[],None,1164,Season,c5719ef0-3dcc-4740-84af-ef600d160bd9,ddd5757d-e6f2-5bdc-9d42-4fe825a2eaed,4.7,302.0,WR3,WR,None,Cooper,Kupp,NFL,d150534e-6a05-587b-b9e3-50ef86602e20,LAR,Los Angeles Rams
4,c7bc600e-e3be-48ff-86cf-3b1d0ad42a5f,[],None,1164,Season,9ba8d5db-3aa3-4600-98f0-406a1e67bb22,540bb60e-ece4-5478-b5b1-4acd97db8102,5.0,209.2,TE1,TE,None,Travis,Kelce,NFL,a6f458f4-5078-56e4-a839-af96f1191314,KC,Kansas City Chiefs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1559,77f23ed4-a5aa-4f69-ad6d-3fb6c537d20b,[],None,1164,Season,ddf60423-df8c-4152-88ae-3c14a03c8639,e2571131-7600-5bc9-878b-946a992c8203,-,0.0,None,RB,None,Bryce,Love,NFL,None,NaN,NaN
1560,786211aa-529b-4615-8602-903993521a63,[],None,1164,Season,7565ecc7-a461-4a04-9766-4176c9fbf6fc,540bb60e-ece4-5478-b5b1-4acd97db8102,-,0.0,None,TE,None,Devin,Funchess,NFL,None,NaN,NaN
1561,78e4047a-a9ff-467f-8ca6-9b260f9360aa,[],None,1164,Season,eceff617-b29b-4615-84ae-46ee823e3f38,30f5b4f6-5306-5722-997c-a2255f44225b,-,0.0,None,QB,None,Blake,Bortles,NFL,None,NaN,NaN
1562,7963a975-5ff7-46c9-9fdf-a9844f4a1aba,[],None,1164,Season,ecb8932a-42c9-4dce-a1bd-e0a0aa21406b,e2571131-7600-5bc9-878b-946a992c8203,-,0.0,None,RB,None,Jacques,Patrick,NFL,None,NaN,NaN


In [7]:
from UD_draft_model.data_processing import read_data

DATA_FOLDER = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model/data'
RANKS_FOLDER = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model\
/data/2022/player_ranks'

# df_ranks = read_data.read_ranks(DATA_FOLDER, [2021, 2022])
# df_lookups = read_data.read_lookups(DATA_FOLDER, [2021, 2022])
df_drafts = read_data.read_drafts(DATA_FOLDER, [2021, 2022])

df_drafts

,appearance_id,created_at,draft_entry_id,number,projection_adp,draft_id,position,team_name,draft_source,draft_title,full_name
0,78a5634d-93aa-4cd9-b1af-dfb829df452c,2021-08-25T01:44:33Z,74b3fdcc-6128-4e7a-ae1a-fda04c1859d3,1,1,d525469e-276a-4cf3-ad07-a268841faea3,RB,Carolina Panthers,NaN,NaN,Christian McCaffrey
1,a971451b-f89e-498f-a6b4-060854880ed6,2021-08-25T01:45:03Z,9489f27e-36b8-457c-bbc7-dd7757051ca9,2,2.1,d525469e-276a-4cf3-ad07-a268841faea3,RB,Minnesota Vikings,NaN,NaN,Dalvin Cook
2,30149020-3a70-47ce-a8e2-b52f0a5770fa,2021-08-25T01:45:06Z,b84b9608-e0c8-4d65-a4ab-7698168e4c9f,3,3.7,d525469e-276a-4cf3-ad07-a268841faea3,RB,New Orleans Saints,NaN,NaN,Alvin Kamara
3,53565b50-b52d-4c4f-bcf7-ee96476f4bbc,2021-08-25T01:45:26Z,76e8b661-e9b0-4e1d-a5b5-59184955a1bb,4,4.3,d525469e-276a-4cf3-ad07-a268841faea3,RB,Tennessee Titans,NaN,NaN,Derrick Henry
4,c5897583-8409-4f22-a2af-e0eb91deaadd,2021-08-25T01:45:32Z,5de796ee-9b9e-4975-bbe6-5abd444c41db,5,5.6,d525469e-276a-4cf3-ad07-a268841faea3,RB,Dallas Cowboys,NaN,NaN,Ezekiel Elliott
...,...,...,...,...,...,...,...,...,...,...,...
10795,b642a6fb-b8d2-4ee3-8456-39a2c64d2364,2022-05-16T00:02:14Z,42c69461-ef81-4273-bd07-d0d742e77158,212,204.5,133d89ab-ba4f-4230-9148-396bee781f5c,RB,Carolina Panthers,sit_and_go,NaN,D'Onta Foreman
10796,d7c817ad-4ca5-47c5-a458-65f1d441e3fd,2022-05-16T00:02:19Z,5a1a0663-903a-4826-a76f-fef1342be0c4,213,201.6,133d89ab-ba4f-4230-9148-396bee781f5c,TE,Tennessee Titans,sit_and_go,NaN,Austin Hooper
10797,57b58b3c-832f-4981-9461-b521b67c836e,2022-05-16T00:02:40Z,87d3be5d-30b2-41bc-b1be-a54e0e02a184,214,214.9,133d89ab-ba4f-4230-9148-396bee781f5c,WR,New England Patriots,sit_and_go,NaN,Nelson Agholor
10798,21e0b083-749d-4624-b09d-0dabc82d737a,2022-05-16T00:02:45Z,6f76916b-cc71-4e64-8415-96372cce4ac7,215,208.5,133d89ab-ba4f-4230-9148-396bee781f5c,WR,Baltimore Ravens,sit_and_go,NaN,Devin Duvernay


In [13]:
drafted_players = list(df_draft['appearance_id'].iloc[0:2])
df_players.loc[~df_players['appearance_id'].isin(drafted_players)]



,appearance_id,badges,lineup_status_id,match_id,match_type,player_id,position_id,adp,season_projected_points,position_rank,position,country,first_name,last_name,sport_id,team_id,abbr,team_name
1,c307679c-8ee9-47e3-882f-f386bc00f510,[],None,1164,Season,19818cc6-e21b-4258-9315-93cb64fb92e7,ddd5757d-e6f2-5bdc-9d42-4fe825a2eaed,2.5,288.3,WR2,WR,None,Ja'Marr,Chase,NFL,ab235cf0-a041-5d36-8241-90a90f0dcb5e,CIN,Cincinnati Bengals
3,c7bc600e-e3be-48ff-86cf-3b1d0ad42a5f,[],None,1164,Season,9ba8d5db-3aa3-4600-98f0-406a1e67bb22,540bb60e-ece4-5478-b5b1-4acd97db8102,4.7,273.0,TE1,TE,None,Travis,Kelce,NFL,a6f458f4-5078-56e4-a839-af96f1191314,KC,Kansas City Chiefs
4,e32a0878-ba15-4fb1-8518-d86663bbaa17,[],None,1164,Season,c5719ef0-3dcc-4740-84af-ef600d160bd9,ddd5757d-e6f2-5bdc-9d42-4fe825a2eaed,4.9,302.0,WR3,WR,None,Cooper,Kupp,NFL,d150534e-6a05-587b-b9e3-50ef86602e20,LAR,Los Angeles Rams
5,8db7e427-3f92-499e-a538-b585effeb21c,[],None,1164,Season,84cf37a2-d9f9-4315-a56a-c003a186023d,ddd5757d-e6f2-5bdc-9d42-4fe825a2eaed,6.3,258.7,WR4,WR,None,Tyreek,Hill,NFL,9153e0a9-da83-5246-ba43-417537f1bcce,MIA,Miami Dolphins
6,1a43386c-88de-4487-b5bb-f59e02899141,[],None,1164,Season,7e6000de-0a9b-4d12-862b-5194ce6d526c,e2571131-7600-5bc9-878b-946a992c8203,7.7,333.1,RB2,RB,None,Austin,Ekeler,NFL,c7b497d4-18b6-522b-abaa-e5c3d24bc021,LAC,Los Angeles Chargers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1559,928c5651-c73c-4805-95c8-75d3d339f726,[],None,1164,Season,b7f19358-0d6f-48fb-a714-66b773e0ebfb,ddd5757d-e6f2-5bdc-9d42-4fe825a2eaed,-,0.0,None,WR,None,Devon,Williams,NFL,None,NaN,NaN
1560,92bf579e-e55f-4e7e-b368-d60e7e057a12,[],None,1164,Season,6f2c6b80-e459-40cb-b79c-7c4c85090213,ddd5757d-e6f2-5bdc-9d42-4fe825a2eaed,-,0.0,None,WR,None,Penny,Hart,NFL,31631011-5902-52f6-ba01-c4c8d8eb3fd9,SEA,Seattle Seahawks
1561,92cad006-95d3-4e37-af92-50a26213d96f,[],None,1164,Season,4bde293b-f493-4076-8b2a-7c836a3a7530,ddd5757d-e6f2-5bdc-9d42-4fe825a2eaed,-,0.0,None,WR,None,Jaylen,Smith,NFL,None,NaN,NaN
1562,92d8561a-be80-4546-847d-b517611c1d4f,[],None,1164,Season,598c709a-e526-4e41-bd54-5ec82ef5b046,be5a7215-24b9-50f9-973c-bd5f344dd964,-,0.0,None,RB,None,Nick,Bellore,NFL,31631011-5902-52f6-ba01-c4c8d8eb3fd9,SEA,Seattle Seahawks


In [15]:
df_drafts.loc[df_drafts['draft_id'] == 'd525469e-276a-4cf3-ad07-a268841faea3']

,appearance_id,created_at,draft_entry_id,number,projection_adp,draft_id,position,team_name,draft_source,draft_title,full_name
0,78a5634d-93aa-4cd9-b1af-dfb829df452c,2021-08-25T01:44:33Z,74b3fdcc-6128-4e7a-ae1a-fda04c1859d3,1,1,d525469e-276a-4cf3-ad07-a268841faea3,RB,Carolina Panthers,NaN,NaN,Christian McCaffrey
1,a971451b-f89e-498f-a6b4-060854880ed6,2021-08-25T01:45:03Z,9489f27e-36b8-457c-bbc7-dd7757051ca9,2,2.1,d525469e-276a-4cf3-ad07-a268841faea3,RB,Minnesota Vikings,NaN,NaN,Dalvin Cook
2,30149020-3a70-47ce-a8e2-b52f0a5770fa,2021-08-25T01:45:06Z,b84b9608-e0c8-4d65-a4ab-7698168e4c9f,3,3.7,d525469e-276a-4cf3-ad07-a268841faea3,RB,New Orleans Saints,NaN,NaN,Alvin Kamara
3,53565b50-b52d-4c4f-bcf7-ee96476f4bbc,2021-08-25T01:45:26Z,76e8b661-e9b0-4e1d-a5b5-59184955a1bb,4,4.3,d525469e-276a-4cf3-ad07-a268841faea3,RB,Tennessee Titans,NaN,NaN,Derrick Henry
4,c5897583-8409-4f22-a2af-e0eb91deaadd,2021-08-25T01:45:32Z,5de796ee-9b9e-4975-bbe6-5abd444c41db,5,5.6,d525469e-276a-4cf3-ad07-a268841faea3,RB,Dallas Cowboys,NaN,NaN,Ezekiel Elliott
...,...,...,...,...,...,...,...,...,...,...,...
211,9b8176b0-ef42-4f68-9d0f-908acc6f556f,2021-08-25T02:34:35Z,5de796ee-9b9e-4975-bbe6-5abd444c41db,212,214.9,d525469e-276a-4cf3-ad07-a268841faea3,WR,Las Vegas Raiders,NaN,NaN,Tyron Johnson
212,04c2313c-7997-4cda-9089-b8f11108f2da,2021-08-25T02:34:39Z,76e8b661-e9b0-4e1d-a5b5-59184955a1bb,213,215.6,d525469e-276a-4cf3-ad07-a268841faea3,QB,Houston Texans,NaN,NaN,Tyrod Taylor
213,5949a3d1-56be-4b3e-b75b-827f30450b94,2021-08-25T02:34:43Z,b84b9608-e0c8-4d65-a4ab-7698168e4c9f,214,206.9,d525469e-276a-4cf3-ad07-a268841faea3,WR,Kansas City Chiefs,NaN,NaN,Demarcus Robinson
214,ca04265c-c408-4d13-869d-101f4931214a,2021-08-25T02:34:49Z,9489f27e-36b8-457c-bbc7-dd7757051ca9,215,211.5,d525469e-276a-4cf3-ad07-a268841faea3,QB,Carolina Panthers,NaN,NaN,Cam Newton


In [38]:
out = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model/UD_draft_model/scratch'
active_drafts = scrape_site.DraftsActive(bearer_token)

df = pd.read_pickle(join(out, 'active_drafts2.pkl'))
df = active_drafts._add_scoring_type(df.copy())

slate_id = df['slate_id'].iloc[0]
scoring_type_id = df['scoring_type_id'].iloc[0]
draft_id = [df['id'].iloc[0]]
refs = scrape_site.ReferenceData(slate_id, scoring_type_id)

draft_detail = scrape_site.DraftsDetail(draft_id, bearer_token)

df_players = refs.create_df_players_master()
df_draft = draft_detail.create_df_drafts()